# Extracting 
We've extracted two datasets from kaggle, one containing 2018 FIFA world cup data on the national squads, and another with data on soccer players from major leagues in the world.

In [1]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
import pymysql
pymysql.install_as_MySQLdb()

# Transform

We've used pandas to get input csv files, create dataframes, and clean the data.

In [3]:
# wc csv file path
csv_wc = "data/2018FIFAWC.csv"

# create wc squads df 
wc_df = pd.read_csv(csv_wc)
wc_df["Player"] = wc_df["Player"].map(lambda x: x.strip('(captain)'))
wc_df = wc_df.drop(columns=["Player Count","Date Of Birth"])
wc_df = wc_df.rename(columns={"Player":"name_","Group":"group_","Squad Number":"squad_number","Position":"position_"})
wc_df.head()

,Team,group_,squad_number,position_,name_,Age,Caps,Goals,Club
0,Egypt,A,1,GK,Essam El-Hadary,45,157,0,Al-Taawoun
1,Egypt,A,2,DF,Ali Gabr,29,20,1,West Bromwich Albion
2,Egypt,A,3,DF,Ahmed Elmohamady,30,77,2,Aston Villa
3,Egypt,A,4,MF,Omar Gaber,26,23,0,Los Angeles FC
4,Egypt,A,5,MF,Sam Morsy,26,4,0,Wigan Athletic


In [4]:
# players csv file path
csv_players = "data/players.csv"

# create players df
players_df = pd.read_csv(csv_players)
players_df = players_df.dropna()
players_df[["National_Kit","Club_Kit","Contract_Expiry"]] = players_df[["National_Kit","Club_Kit","Contract_Expiry"]].astype(int)
players_df["Height"] = players_df["Height"].map(lambda x: x.rstrip('cm'))
players_df["Weight"] = players_df["Weight"].map(lambda x: x.rstrip('kg'))
players_df = players_df.rename(columns={"Name":"name_"})
players_df.head()

,name_,Nationality,National_Position,National_Kit,Club,Club_Position,Club_Kit,Club_Joining,Contract_Expiry,Rating,...,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,GK_Positioning,GK_Diving,GK_Kicking,GK_Handling,GK_Reflexes
0,Cristiano Ronaldo,Portugal,LS,7,Real Madrid,LW,7,07/01/2009,2021,94,...,90,81,76,85,88,14,7,15,11,11
1,Lionel Messi,Argentina,RW,10,FC Barcelona,RW,10,07/01/2004,2018,93,...,88,89,90,74,85,14,6,15,11,8
2,Neymar,Brazil,LW,10,FC Barcelona,LW,11,07/01/2013,2021,92,...,77,79,84,81,83,15,9,15,9,11
3,Luis Suárez,Uruguay,LS,9,FC Barcelona,ST,9,07/11/2014,2021,92,...,86,86,84,85,88,33,27,31,25,37
4,Manuel Neuer,Germany,GK,1,FC Bayern,GK,1,07/01/2011,2021,92,...,16,14,11,47,11,91,89,95,90,89


In [5]:
columns_to_drop = [
    'Work_Rate', 'Weak_foot', 'Skill_Moves',
    'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
    'Standing_Tackle', 'Aggression', 'Reactions', 'Attacking_Position',
    'Interceptions', 'Vision', 'Composure', 'Crossing', 'Short_Pass',
    'Long_Pass', 'Acceleration', 'Speed', 'Stamina', 'Strength', 'Balance',
    'Agility', 'Jumping', 'Heading', 'Shot_Power', 'Finishing',
    'Long_Shots', 'Curve', 'Freekick_Accuracy', 'Penalties', 'Volleys',
    'GK_Positioning', 'GK_Diving', 'GK_Kicking', 'GK_Handling',
    'GK_Reflexes','Club_Joining','Birth_Date','Preffered_Position'
]

players_df_clean = players_df.drop(columns=columns_to_drop)
players_df_clean.head()

,name_,Nationality,National_Position,National_Kit,Club,Club_Position,Club_Kit,Contract_Expiry,Rating,Height,Weight,Preffered_Foot,Age
0,Cristiano Ronaldo,Portugal,LS,7,Real Madrid,LW,7,2021,94,185,80,Right,32
1,Lionel Messi,Argentina,RW,10,FC Barcelona,RW,10,2018,93,170,72,Left,29
2,Neymar,Brazil,LW,10,FC Barcelona,LW,11,2021,92,174,68,Right,25
3,Luis Suárez,Uruguay,LS,9,FC Barcelona,ST,9,2021,92,182,85,Right,30
4,Manuel Neuer,Germany,GK,1,FC Bayern,GK,1,2021,92,193,92,Right,31


# Load
We've used sqlalchemy and pymysql to load our cleaned dataframes to our production database.

In [6]:
# create connection to db
connection_string = "root:root@127.0.0.1/soccer_data"
engine = create_engine("mysql://{}".format(connection_string))

In [7]:
# create table for wc data
wc_df.to_sql(name='world_cup_squads', con=engine, if_exists='append', index=False)

In [8]:
# create table for players data
players_df_clean.to_sql(name='players', con=engine, if_exists='append', index=False)

In [9]:
# confirm wc table has been added
pd.read_sql_query('select * from world_cup_squads', con=engine).head()

,id,team,group_,squad_number,position_,name_,age,caps,goals,club
0,1,Egypt,A,1,GK,Essam El-Hadary,45,157,0,Al-Taawoun
1,2,Egypt,A,2,DF,Ali Gabr,29,20,1,West Bromwich Albion
2,3,Egypt,A,3,DF,Ahmed Elmohamady,30,77,2,Aston Villa
3,4,Egypt,A,4,MF,Omar Gaber,26,23,0,Los Angeles FC
4,5,Egypt,A,5,MF,Sam Morsy,26,4,0,Wigan Athletic


In [10]:
# confirm players table has been added
pd.read_sql_query('select * from players', con=engine).head()

,id,name_,nationality,national_Position,national_Kit,club,club_position,club_kit,contract_expiry,rating,height,weight,preffered_foot,age
0,1,Cristiano Ronaldo,Portugal,LS,7,Real Madrid,LW,7,2021,94,185,80,Right,32
1,2,Lionel Messi,Argentina,RW,10,FC Barcelona,RW,10,2018,93,170,72,Left,29
2,3,Neymar,Brazil,LW,10,FC Barcelona,LW,11,2021,92,174,68,Right,25
3,4,Luis Suárez,Uruguay,LS,9,FC Barcelona,ST,9,2021,92,182,85,Right,30
4,5,Manuel Neuer,Germany,GK,1,FC Bayern,GK,1,2021,92,193,92,Right,31
